# **Implémentation d'un modèle de scoring**
## **Notebook 6/6 - Assemblage des données TEST pour le dashboard**
*Sofia Chevrolat (Novembre 2020)*
___
Cette étude vise à réaliser un modèle permettant de prédire le risque de défaut de remboursement d'un prêt pour un client donné. Ce modèle doit être basé sur une variété de données (personnelles, en provenance de différentes sources bancaires, etc.).

Ce modèle est destiné à être servi via une API, elle-même appelée via un dashboard interactif. Le modèle devra donc être exporté une fois finalisé.
___
_**Remerciements**:<br>
Merci à mon compagnon [J. Duplan](https://www.linkedin.com/in/julian-duplan-64844a41/) pour les discussions intéressantes.<br>
Merci également à mon mentor [Samia Drappeau](https://www.linkedin.com/in/samiadrappeau) pour les échanges d'idées, les conseils et les encouragements !_
___


Ce notebook est organisé comme suit:

**0. Mise en place**
- 0.1 Chargement des librairies et fonctions utiles
- 0.2 Chargement du jeu de données
- 0.3 Séparation du jeu de données
- 0.3 Chargement du modèle
    
**1. Exploitation du modèle : Prédictions sur de nouvelles données**
- 1.1 Transformation des données pour consommation par le modèle
- 1.2 Prédictions parle modèle
- 1.3 Export des prédictions

___
### 0. MISE EN PLACE

Dans cette première étape, le cadre de travail est mis en place, c'est-à-dire :
- Les librairies et packages Python nécessaires sont chargés
- Les fonctions utiles sont définies
- Le jeu de données est chargé
___

___
#### 0.1 CHARGEMENT DES LIBRAIRIES ET FONCTIONS UTILES

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from joblib import load

In [3]:
from sys import path
path.append("./Resources/functions")

import helper_functions as hf

____
#### 0.2 CHARGEMENT DU JEU DE DONNÉES

Afin de démontrer toute la chaîne d'exploitation du modèle, cette partie va utiliser le jeu de données TEST application_test.csv.

Ces données n'ayant été utilisées dans aucune des différentes étapes d'élaboration du modèle, elles permettent de montrer l'utilisation du modèle avec de nouvelles données.

In [4]:
app_test = pd.read_csv("./Resources/datasets/origin/application_test.csv")

bureau_balance = pd.read_csv("./Resources/datasets/origin/bureau_balance.csv")

bureau = pd.read_csv("./Resources/datasets/origin/bureau.csv")

card = pd.read_csv("./Resources/datasets/origin/credit_card_balance.csv")

installments = pd.read_csv("./Resources/datasets/origin/installments_payments.csv")

cash = pd.read_csv("./Resources/datasets/origin/POS_CASH_balance.csv")

prev_app = pd.read_csv("./Resources/datasets/origin/previous_application.csv")

___

#### 0.3 SÉPARATION DU JEU DE DONNÉES

Le jeu de données est séparé en deux :
- données pour lesquelles le credit score aura déjà été calculé, qui seront utilisées au démarrage du dashboard, pour être parcourues (_browse).
- données pour lesquelles le credit score sera calculé en live par le dashboard (_new).

In [5]:
app_test_browse = app_test[:int(len(app_test)/2)]

bureau_browse = bureau[bureau["SK_ID_CURR"].isin(app_test_browse["SK_ID_CURR"])]
bureau_balance_browse = bureau_balance[bureau_balance["SK_ID_BUREAU"].isin(bureau_browse["SK_ID_BUREAU"])]
card_browse = card[card["SK_ID_CURR"].isin(app_test_browse["SK_ID_CURR"])]
installments_browse = installments[installments["SK_ID_CURR"].isin(app_test_browse["SK_ID_CURR"])]
cash_browse = cash[cash["SK_ID_CURR"].isin(app_test_browse["SK_ID_CURR"])]
prev_app_browse = prev_app[prev_app["SK_ID_CURR"].isin(app_test_browse["SK_ID_CURR"])]

In [6]:
app_test_new = app_test[int(len(app_test)/2):]

bureau_new = bureau[bureau["SK_ID_CURR"].isin(app_test_new["SK_ID_CURR"])]
bureau_balance_new = bureau_balance[bureau_balance["SK_ID_BUREAU"].isin(bureau_new["SK_ID_BUREAU"])]
card_new = card[card["SK_ID_CURR"].isin(app_test_new["SK_ID_CURR"])]
installments_new = installments[installments["SK_ID_CURR"].isin(app_test_new["SK_ID_CURR"])]
cash_new = cash[cash["SK_ID_CURR"].isin(app_test_new["SK_ID_CURR"])]
prev_app_new = prev_app[prev_app["SK_ID_CURR"].isin(app_test_new["SK_ID_CURR"])]

___
#### 0.4 CHARGEMENT DU MODÈLE

In [7]:
model = load('../lgbm_trained_model_whole_dataset.joblib')

___
### 1. EXPLOITATION DU MODÈLE : PRÉDICTIONS SUR DE NOUVELLES DONNÉEs

___
#### 1.1 TRANSFORMATION DES DONNÉES POUR CONSOMMATION PAR LE MODÈLE

In [8]:
model_ready_df = hf.transform_data(app_test_browse, 
                                   bureau_browse,                                   
                                   bureau_balance_browse,
                                   card_browse, 
                                   cash_browse, 
                                   installments_browse,
                                   prev_app_browse)

model_ready_df.shape

Original Memory Usage: 0.02 gb.
New Memory Usage: 0.01 gb.
Original Memory Usage: 0.15 gb.
New Memory Usage: 0.08 gb.
There are 0 columns with greater than 90% missing values.
Original Memory Usage: 0.04 gb.
New Memory Usage: 0.01 gb.
There are 6 columns with greater than 90% missing values.
Original Memory Usage: 0.05 gb.
New Memory Usage: 0.03 gb.
There are 0 columns with greater than 90% missing values.
Original Memory Usage: 0.06 gb.
New Memory Usage: 0.03 gb.
There are 0 columns with greater than 90% missing values.
Original Memory Usage: 0.07 gb.
New Memory Usage: 0.04 gb.
There are 0 columns with greater than 90% missing values.


(24372, 141)

___
#### 1.2 PRÉDICTIONS PAR LE MODÈLE

In [9]:
# Predictions on the test data
credit_score_predictions = model.predict_proba(model_ready_df)[:, 1]

In [10]:
predictions_df = pd.DataFrame(data=credit_score_predictions*100)\
                   .rename(columns={0:"Credit Score"})

In [11]:
predictions_df = pd.concat([app_test_browse["SK_ID_CURR"], predictions_df], axis=1)

___
#### 1.3 EXPORT DES DONNÉES ET PRÉDICTIONS

In [12]:
# Exporting predictions
predictions_df.to_csv("./Resources/datasets/assembled/dashboard/browse/predictions_test.csv")

In [13]:
# Removing anomaly from already processed data
app_test_browse['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

# Exporting browse data
app_test_browse.to_csv("./Resources/datasets/assembled/dashboard/browse/application_test.csv")
bureau_browse.to_csv("./Resources/datasets/assembled/dashboard/browse/bureau.csv")
bureau_balance_browse.to_csv("./Resources/datasets/assembled/dashboard/browse/bureau_balance.csv")
card_browse.to_csv("./Resources/datasets/assembled/dashboard/browse/card.csv")
installments_browse.to_csv("./Resources/datasets/assembled/dashboard/browse/installments.csv")
cash_browse.to_csv("./Resources/datasets/assembled/dashboard/browse/cash.csv")
prev_app_browse.to_csv("./Resources/datasets/assembled/dashboard/browse/prev_app.csv")

In [14]:
# Exporting new data
app_test_new.to_csv("./Resources/datasets/assembled/dashboard/new/application_test.csv")
bureau_new.to_csv("./Resources/datasets/assembled/dashboard/new/bureau.csv")
bureau_balance_new.to_csv("./Resources/datasets/assembled/dashboard/new/bureau_balance.csv")
card_new.to_csv("./Resources/datasets/assembled/dashboard/new/card.csv")
installments_new.to_csv("./Resources/datasets/assembled/dashboard/new/installments.csv")
cash_new.to_csv("./Resources/datasets/assembled/dashboard/new/cash.csv")
prev_app_new.to_csv("./Resources/datasets/assembled/dashboard/new/prev_app.csv")